In [25]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from IPython.display import display, HTML
import time

In [4]:
df1 = pd.read_csv("zomato_data.csv")
df2 = pd.read_csv("Geographical Coordinates.csv")

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   online_order               51717 non-null  object
 1   book_table                 51717 non-null  object
 2   rate                       43942 non-null  object
 3   votes                      51717 non-null  int64 
 4   rest_type                  51490 non-null  object
 5   dish_liked                 23639 non-null  object
 6   cuisines                   51672 non-null  object
 7   approx_costfor_two_people  51371 non-null  object
 8   listed_intype              51717 non-null  object
 9   listed_incity              51717 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.9+ MB


In [17]:
df1.head()

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity
0,Yes,Yes,4.1,775,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari
1,Yes,No,4.1,787,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet,Banashankari
2,Yes,No,3.8,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet,Banashankari
3,No,No,3.7,88,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet,Banashankari
4,No,No,3.8,166,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet,Banashankari


In [14]:
df1["rate"] = df1["rate"].str.replace("/5","")

In [22]:
df1["rate"] = df1["rate"].replace(['-',"NEW"],np.nan)

In [23]:
df1["rate"] = pd.to_numeric(df1["rate"])

In [26]:
df1["rate"].fillna(df1["rate"].median(), inplace=True)

In [34]:
print(df1['rate'].isnull().sum())

0


In [35]:
df1['approx_costfor_two_people'] = df1['approx_costfor_two_people'].astype(str).str.replace(',', '')


In [36]:
df1['approx_costfor_two_people'] = df1['approx_costfor_two_people'].replace('nan', np.nan)

In [37]:
df1['approx_costfor_two_people'] = pd.to_numeric(df1['approx_costfor_two_people'])

In [38]:
df1['approx_costfor_two_people'].fillna(df1['approx_costfor_two_people'].median, inplace=True)

In [39]:
print(df1['approx_costfor_two_people'].isnull().sum())

0


In [46]:
df1["dish_liked"].fillna("Not Available", inplace=True)
df1["cuisines"].fillna("Other",inplace=True)
df1["rest_type"].fillna("Unknown",inplace=True)

In [49]:
df1["votes"].fillna(df1["votes"].median,inplace=True)

In [51]:
print(df1['votes'].isnull().sum())

0


In [52]:
binary ={"Yes":1, "No":0}
df1["online_order"] = df1["online_order"].map(binary)
df1["book_table"] = df1["book_table"].map(binary)

In [61]:
df1["rate"]=df1["rate"].astype(float)
df1["votes"]=df1["votes"].astype(int)
df1["approx_costfor_two_people"]=df1["approx_costfor_two_people"].astype(int)

In [56]:
df1['approx_costfor_two_people'] = df1['approx_costfor_two_people'].replace('nan', np.nan)


In [62]:
print(df1.isnull().sum().sum())


0


In [63]:
df1.describe()

,online_order,book_table,rate,votes,approx_costfor_two_people
count,51717.000000,51717.000000,51717.000000,51717.000000,51717.000000
mean,0.588665,0.124698,3.700362,283.697527,554.391689
std,0.492080,0.330379,0.395391,803.838853,437.563723
min,0.000000,0.000000,1.800000,0.000000,40.000000
25%,0.000000,0.000000,3.500000,7.000000,300.000000
50%,1.000000,0.000000,3.700000,41.000000,400.000000
75%,1.000000,0.000000,3.900000,198.000000,650.000000
max,1.000000,1.000000,4.900000,16832.000000,6000.000000


In [64]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   online_order               51717 non-null  int64  
 1   book_table                 51717 non-null  int64  
 2   rate                       51717 non-null  float64
 3   votes                      51717 non-null  int32  
 4   rest_type                  51717 non-null  object 
 5   dish_liked                 51717 non-null  object 
 6   cuisines                   51717 non-null  object 
 7   approx_costfor_two_people  51717 non-null  int32  
 8   listed_intype              51717 non-null  object 
 9   listed_incity              51717 non-null  object 
dtypes: float64(1), int32(2), int64(2), object(5)
memory usage: 3.6+ MB


In [65]:
merged_data = pd.merge(df1, df2, on='listed_incity', how='left')


In [66]:
merged_data.head()

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity,Latitude,Longitude
0,1,1,4.1,775,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari,12.939333,77.553982
1,1,0,4.1,787,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet,Banashankari,12.939333,77.553982
2,1,0,3.8,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet,Banashankari,12.939333,77.553982
3,0,0,3.7,88,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet,Banashankari,12.939333,77.553982
4,0,0,3.8,166,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet,Banashankari,12.939333,77.553982


In [ ]:
df1.shape #1. What is the shape of the given dataset?

(51717, 10)

In [ ]:
north_indian_count = df1[df1['cuisines'].str.contains('North Indian', na=False)].shape[0] #2. How many restaurants serve North Indian cuisine?
north_indian_count

21085

In [ ]:
common_cuisine = df1['cuisines'].str.split(', ').explode().value_counts().idxmax() #3. What cuisine is most commonly offered by restaurants in Bangalore?
common_cuisine

'North Indian'

In [71]:
highest_avg_cost_locality = df1.groupby('listed_incity')['approx_costfor_two_people'].mean().idxmax()
highest_avg_cost_locality

'Church Street'

In [77]:
high_vote = df1[df1['votes'] > 1000] #5. Which restaurant type has the top rating with over 1000 votes?
top_restaurant_type = high_vote.groupby('rest_type')['rate'].mean().idxmax()
top_restaurant_type

'Bakery'

In [78]:
min_cost = df1['approx_costfor_two_people'].min() #6. How much does it cost at minimum to eat out in Bangalore?
min_cost

40

In [79]:
print(df1['approx_costfor_two_people'].unique())


[ 800  300  600  700  550  500  450  650  400  900  200  750  150  850
  100 1200  350  250  950 1000 1500 1300  199   80 1100  160 1600  230
  130   50  190 1700 1400  180 1350 2200 2000 1800 1900  330 2500 2100
 3000 2800 3400   40 1250 3500 4000 2400 2600  120 1450  469   70 3200
   60  560  240  360 6000 1050 2300 4100 5000 3700 1650 2700 4500  140]


In [80]:
banashankari_orders = df1[(df1['listed_incity'] == 'Banashankari') & (df1['online_order'] == 1)]


In [81]:
banashankari_online_count = banashankari_orders.shape[0]
banashankari_online_count

546

In [83]:
total_online_orders = df1[df1['online_order'] == 1].shape[0]
total_online_orders

30444

In [ ]:
filtered_df = df1[(df1['votes'] > 500) & (df1['rate'] < 3.0)] #8. Which locality has the most restaurants with over 500 votes and a rating below 3.0?
top_locality = filtered_df['listed_incity'].value_counts().idxmax()
top_locality

'Brookefield'

In [89]:
locality_diversity = df1.groupby('listed_incity')['rest_type'].nunique()

best_expansion_locality = locality_diversity.idxmax()
best_expansion_locality

'BTM'

In [100]:
df1.head()

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity
0,1,1,4.1,775,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari
1,1,0,4.1,787,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet,Banashankari
2,1,0,3.8,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet,Banashankari
3,0,0,3.7,88,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet,Banashankari
4,0,0,3.8,166,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet,Banashankari


In [104]:
north_chinese_restaurant = df1[df1['cuisines'].str.contains('North Indian') & df1['cuisines'].str.contains('Chinese', na=False)]

average_rating = north_chinese_restaurant['rate'].mean()
average_rating


3.5919080217658412

In [105]:
# Group by restaurant type and calculate the average rating for each type
avg_rating_by_rest_type = df1.groupby('rest_type')['rate'].mean()

# Find the restaurant type with the lowest average rating
lowest_rating_rest_type = avg_rating_by_rest_type.idxmin()
lowest_rating = avg_rating_by_rest_type.min()

print(f"Zomato should focus on improving the {lowest_rating_rest_type} restaurant type, which has an average rating of {lowest_rating:.2f}")


Zomato should focus on improving the Dessert Parlor, Kiosk restaurant type, which has an average rating of 3.00


In [108]:
# Keep only Italian restaurants
italian_df = merged_data[merged_data['cuisines'].str.contains('Italian', na=False)]
italian_df.head()


,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity,Latitude,Longitude
2,1,0,3.8,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet,Banashankari,12.939333,77.553982
7,1,1,4.6,2556,"Casual Dining, Cafe","Farmhouse Pizza, Chocolate Banana, Virgin Moji...","Pizza, Cafe, Italian",600,Cafes,Banashankari,12.939333,77.553982
8,1,0,4.0,324,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,Cafes,Banashankari,12.939333,77.553982
9,1,0,4.2,504,Cafe,"Waffles, Pasta, Coleslaw Sandwich, Choco Waffl...","Cafe, Mexican, Italian, Momos, Beverages",550,Cafes,Banashankari,12.939333,77.553982
11,1,1,4.2,150,Cafe,"Mocktails, Peri Fries, Lasagne, Pizza, Chicken...","Cafe, Italian, Continental",600,Cafes,Banashankari,12.939333,77.553982


In [111]:
import folium
from IPython.display import IFrame

# Initialize map centered on Bangalore
bangalore_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)

# Add a purple marker for each Italian restaurant
for index, row in italian_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['listed_incity']}<br>{row['cuisines']}",
        icon=folium.Icon(color='purple')
    ).add_to(bangalore_map)

# Save the map
bangalore_map.save("italian_restaurants_map.html")


In [127]:
# Remove rows with missing Latitude or Longitude (since Folium can't plot them)
merged_data = merged_data.dropna()


In [128]:
# Add a marker for each restaurant
for index, row in merged_data.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=f"{row['listed_incity']} - {row['cuisines']}"
    ).add_to(bangalore_map)


In [129]:
merged_data[['Latitude', 'Longitude']].isnull().sum()


Latitude     0
Longitude    0
dtype: int64

In [131]:
# Save map to HTML file
bangalore_map.save("restaurant_density.html")


In [133]:
# Embed the saved HTML map inside the notebook
IFrame('restaurant_density.html', width=900, height=600)
